<a href="https://colab.research.google.com/github/GiovanniManiezzo/eeg_signal_processing/blob/main/processamento_eeg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne

In [ ]:
!pip install pymatreader

In [ ]:
#core class - imports the files information
from joblib import load as jLoad
import numpy as np
from numpy import genfromtxt, array, savetxt, argmax
from numpy.random import randint
from pickle import load as pLoad
from mne.io import read_raw_edf, read_raw_eeglab
from os import listdir
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix

#sem os dois imports de classes internas ao projeto


'''
    Cria uma matriz EEG de dimensões t x c x l, onde 't' é o número de ensaios, 'c' é o número de canais e 'l' é o tamanho da gravação.

    Parameters
    ----------
    directory : str
        Nome do diretório que contêm os ensaions do sujeito que deve ser agrupado em uma matriz singular.
        
    fix_length : bool, optional
        Verdadeiro se os ensaios tem tamanhos diferentes, falso caso contrário. O valor padrão é falso.
    length : int, optional
        Tamanho a ser usado em todos os ensaios. Isso deve ser especificado se o "fix_length" for True. O valor padrão é None.
        
    row_channels : bool, optional
        Veradeiro se as linhas de cada ensaio corresponderem aos canais, falso caso contrário. O valor padrão é True.

    Raises
    ------
    TypeError
        Se os parâmetros ou arquivos possuem tipo não suportado.
    RuntimeError
        Se um erro ocorrer para prevenir o funcionamento correto.

    Returns
    -------
    eeg_matrix : ndarray
        Matriz EEG de dimensões t x c x l.

    '''

def create_eeg_matrix(directory, fix_length=False, length=None, row_channels=True):
    
    ## Validaçõe iniciais dos valores de entrada ##
    if type(directory) != str:
        raise TypeError(f"'directory' deve ser do tipo 'str', tipo recebido: '{type(directory).__name__}'")
    if type(fix_length) != bool:
        raise TypeError(f"'fix_length' deve ser do tipo 'bool', tipo recebido: '{type(fix_length).__name__}'")
    if fix_length:
        if (type(length) != int and type(length) != float) or (int(length) != length):
            raise TypeError(f"'length' deve ser do tipo 'int', tipo recebido: '{type(length).__name__}'")
        if length < 1:
            raise TypeError(f"'length' deve ser maior que '0', valor recebido: {length}")
        length = int(length)
    
    if type(row_channels) != bool:
        raise TypeError(f"'row_channels' deve ser do tipo 'bool', tipo recebido: '{type(row_channels).__name__}'")


        ## Listando o número de amostras ##
    try:
      files = os.listdir(f"{gdrive_path}/{trial}")
      print("Trial atual:" + trial)
    except:
        raise

    eeg_matrix = []
    trial_length = None
    ## Validaçõe dos arquivos no diretório de cada amostra ##
    if len(trials) == 0:
      raise RuntimeError("Diretório vazio não pode ser convertido em uma matriz EEG")

    for eeg_file in files:
      
      if "." not in eeg_file:
        raise TypeError(f"File type of '{eeg_file}' not found")

      file_type = eeg_file.split(".")[-1] 

      if file_type == "set":
      #Necessário deixar apenas os arquivos eeg dentro da pasta de amostra
      #if file_type not in ["sav", "csv", "dat", "edf", "set", "fdt"]:
      #  raise TypeError(f"Tipo de arquivo '{trial}' não suprotado. TIpos suportados: sav, csv, dat, edf")
      
        # Verifica o tipo do arquivo e realiza sua importação
        try:
       
          raw_data = read_raw_eeglab(f"{gdrive_path}/{trial}/{eeg_file}")
          data = array(raw_data.get_data())
          print(np.shape(data))

        except:
          raise

        # Verifica condição de parâmetro de entrada inicial
        if not row_channels:
          data = data.T

        if fix_length:
          if len(data[0]) < length:
            raise RuntimeError(f"Todas as amostras deve ter pelo menos {length} de tamanho")
          data = data[:,:length]

          #teste de conteudo na variavel data
        else:
          if  trial_length == None:
            trial_length = len(data[0])
          #else:
            #if len(data[0]) != trial_length:
            #  raise RuntimeError("Todas as amostras devem ter o mesmo tamanho")
        
        eeg_matrix.append(data)
         #print(raw_data.info.get('nchan'))
  
    return array(eeg_matrix, order='A', dtype= float)


In [ ]:
#time windowing class
'''
    Segmenta uma matriz EEG em janelas de tempo específicas

    Parameters
    ----------
    eeg_matrix : ndarray
        Uma matriz EEG de dimensções t x c x l, onde 't' corresponde ao número de ensaios, 'c' ao número de canais e 'l' é o tamanho da gravação. 
    frequency : int
        Frequência de amostragem utilziada para a gravação da matriz EEG.
    time_windows : list
        Janela de tempo, em segundos, na qual os ensaios foram segmentados
    start_type : str, optional
        Método para selecionar o início da segmentação, isso pode ser: 'begin' (segmentação começa no começo da gravação), 'random' (segmentação começa de um ponto aleatório da gravação), 
        ou 'custom' (permite a definição do momento exato para começar a segmentação). O valor padrão é 'begin'.
    start_time : list, optional
        Especifica o ponto, em segundos, de iníco da segmentação de cada ensaio. Pode ser especificado apenas se 'start_type' for 'custom'. O valor padrão é None.
    channels : list, optional
        Lista de canais para usar na segmentação, se None, então todos os canais são usados. O valor padrão é None.
    verbose : bool, optional
        True para receber informação durante o processo de segmentação, False caso contrário. O valor padrão é Ture.

    Raises
    ------
    TypeError
        Se os parâmetros tem tipos não suportados.
    RuntimeError
        Se ocorrer um erro que impede o funcionamento.

    Returns
    -------
    time_segmented_matrices : list
        Lista com as matrizes segmentadas correspondentes a cada janela de tempo.
'''

def time_windowing(eeg_matrix, frequency, time_windows, start_type="begin", start_time=None, channels=None, verbose=True):
  
    # Validações da matriz de entrada: eeg_matrix
    if type(eeg_matrix).__name__ != "ndarray" and type(eeg_matrix) != list:
      raise TypeError(f"'eeg_matrix' must be 'ndarray', received: '{type(eeg_matrix).__name__}'")
    if type(eeg_matrix) == list:
     eeg_matrix = array(eeg_matrix)
    if "float" not in eeg_matrix.dtype.name and "int" not in eeg_matrix.dtype.name:
      raise TypeError(f"All elements of 'eeg_matrix' must be numbers")
    if len(eeg_matrix.shape) != 3:
       raise TypeError(f"'eeg_matrix' must be an matrix of dimensions t x c x l, received: {eeg_matrix.shape}")

    #Validações do parâmetro de entrada: frequency (frequência de amostragem)
    if (type(frequency) != int and type(frequency) != float) or (int(frequency) != frequency):
        raise TypeError(f"'frequency' must be 'int', received: '{type(frequency).__name__}'")
    if frequency < 1:
        raise TypeError(f"'frequency' must be greater than 0, received: {frequency}")
    frequency = int(frequency)

    #Validações do parâmetro de entrada: time_windows (janela de tempo)
    if type(time_windows).__name__ != "ndarray" and type(time_windows) != list:
        raise TypeError(f"'time_windows' must be list of numbers, received: '{type(time_windows).__name__}'")
    if type(time_windows) == list:
        time_windows = array(time_windows)
    if "float" not in time_windows.dtype.name and "int" not in time_windows.dtype.name:
        raise TypeError(f"All elements of 'time_windows' must be numbers")
    for time_window in time_windows:
        if not (time_window > 0):
            raise TypeError(f"All time windows must be greater than 0, received: {time_window}")

    #Validações do parâmetro de entrada: start_type (tipo de início da janela de tempo)
    if start_type not in ["begin", "random", "custom"]:
        raise TypeError(f"Invalid 'start_type' must be 'begin', 'random', or 'custom', received: '{start_type}'")
    if start_type == "custom":
        if type(start_time).__name__ != "ndarray" and type(start_time) != list:
            raise TypeError(f"'start_time' must be list of numbers, received: '{type(start_time).__name__}'")
     #   if not (start_time > 0):
     #       raise TypeError(f"'start_time' must be greater than 0, received: {start_time}")
     #   start = int(start_time*frequency)

    #Validações do parâmetro de entrada: channles (canais que serão seguimentados)
    if type(channels) != list and channels != None:
        raise TypeError(f"'channels' must be list of integers, received: '{type(channels).__name__}'")
    if channels != None:
        for channel in channels:
            if type(channel) != int:
                raise TypeError(f"'channels' must be list of integers, received: '{type(channel).__name__}' inside the list")
            if channel < 0:
                raise TypeError(f"All channels must be greater or equal to 0, received: {channel}")    

    #Validações do parâmetro de entrada: verbose
    if type(verbose) != bool:
      raise TypeError(f"'verbose' must be 'bool', received: '{type(verbose).__name__}'")

    #Validações de janela de tempo
    recording_length = len(eeg_matrix[0,0])
    total_recording_time = recording_length/frequency

    for time in time_windows:
      if time > total_recording_time:
        raise RuntimeError(f"All time windows must be less than or equal to {total_recording_time} seconds because it is the maximum recording time available in the input EEG matrix")
        

    #Segmentação do sinal em janelas de tempo
    time_segmented_matrices = []

    #apenas uma iteração
    for time in time_windows:
      #if verbose: print(f"Início time_windowing")
      time_length = int(time*frequency)

      time_segmented_matrix = []

      #apenas uma iteração
      for trial in range(len(eeg_matrix)):
        #print(len(eeg_matrix))
        #print(trial)
        
        if(time+start_time[trial]) > total_recording_time:
          raise RuntimeError(f"All time windows must be less than or equal to {total_recording_time-start_time[trial]} seconds because it is the maximum recording time available in the input EEG matrix minus the specified start time")
        #Deve ser um int pois vai ser utilizado como index 
        start = int(start_time[trial]*frequency)
        try:
          if channels == None:
              if start_type == "begin":
                segmented_trial = eeg_matrix[trial][:,:time_length]
              elif start_type == "random":
                print(f"Recording {trial} and {start_time[trial]} in segmented_trial")
                random_start =  randint(0, recording_length-time_length+1)
                segmented_trial = eeg_matrix[trial][:,random_start:random_start+time_length]
              else:
                #Start_type = 'Custom', utiliza o start_time refente ao trial
                segmented_trial = eeg_matrix[trial][:,start:start+time_length]

          #Se os canais forem cortados de forma diferente
          else:
            segmented_trial = []
            try:
                if start_type == "random":        
                  random_start =  randint(0, recording_length-time_length+1)
                for channel in channels:
                    if start_type == "begin":
                      segmented_trial.append(eeg_matrix[trial][channel,:time_length])
                    elif start_type == "random":        
                      segmented_trial.append(eeg_matrix[trial][channel,random_start:random_start+time_length])
                    else:
                      segmented_trial.append(eeg_matrix[trial][channel,start:start+time_length])
            except:
              raise
            segmented_trial = array(segmented_trial)
          
        except:
          raise

        #Registro de sucesso da segmentação
        time_segmented_matrix.append(segmented_trial)
      # if verbose: print(f"Amostra {trial} segmentada corretamente")

      time_segmented_matrices.append(array(time_segmented_matrix))
      if verbose: print(f"Fim do método")
      #if verbose: print(f"{'*'*49}\n")
    
    return time_segmented_matrix








In [ ]:
#padding e restauração do sinal (utilizado no método dwt)
from pywt import Wavelet
from math import floor, ceil
from numpy import concatenate, flipud, zeros, convolve, array

'''
    Aplica um preenchimento simétrico do tamanho especificado para o sinal de entrada.
    OBS: técnica de zero-pad anterior a aplicação de uma transformada, adicionar zeros para preencher o sinal.
    Zero padding allows one to use a longer FFT, which will produce a longer FFT result vector.

    Parametros
    ----------
    signal : ndarray
        O sinal a ser preenchido.
    size : int, opcional
        O tamanho do preenchimento o qual corresponde ao tamanho do filtro. O valor padrão é 8. 
      
    Retornos
    -------
    padded_signal : ndarray
        Sinal preenchido.

    '''
def padding_symmetric(signal, size=1):

  padded_signal = concatenate([flipud(signal[:size]), signal, flipud(signal[-size:])])

  return padded_signal

'''
    Restaura o sinal para seu tamanho original usando o filtro de reconstrução.

    Parameters
    ----------
    signal : ndarray
        O signal que será restaurado.
    reconstruction_filter : list
        O filtro de reconstrução que será utilizado para restaurar o sinal.
    real_len : int
        O comprimento real do sinal.

    Returns
    -------
    restored_signal : ndarray
        O sinal restaurado de comprimento especificado.

    '''
def restore_signal(signal, reconstruction_filter, real_len):

  restored_signal = zeros(2 * len(signal) + 1)
  for i in range(len(signal)):
    restored_signal[i * 2 + 1] = signal[i]
  #print(f'real_len = {real_len}')
  restored_signal = convolve(restored_signal, reconstruction_filter)
  restored_len = len(restored_signal)
  exceed_len = (restored_len - real_len) / 2
  restored_signal = restored_signal[int(floor(exceed_len)):(restored_len - int(ceil(exceed_len)))]

  return restored_signal


In [ ]:
#método dwt - extrai as características do sinal após o método time_windowing
from pywt import Wavelet
from math import floor, ceil
'''
    Aplica uma Transformada Wavelet Discreta no sinal.
    
    Parameters
    ----------
    signal : ndarray
        O sinal no qual a DWT será aplicada.
    level : int, optional
        Os níveis de decomposição para a DWT. O padrão é 3.
    mother_wavelet : str, optional
        A wavelet-mãe que será usada na DWT. A padrão é "db4".

    Returns
    -------
    restored_approx_coeff : list
        Armazena os coeficientes de aproximação.
    restored_detail_coeff : list
        Armazena os coeficientes detalhados.

    '''
def dwt(signal, level=3, mother_wavelet='db4'):
  if type(signal).__name__ != 'ndarray' and type (signal) != list:
    raise TypeError(f"'signal' must be 'ndarray', received: '{type(signal).__name__}'")
  if type(signal) == list:
        signal = array(signal)
  if "float" not in signal.dtype.name and "int" not in signal.dtype.name:
        raise TypeError(f"All elements of 'signal' must be numbers")
  if type(level) != int:
        raise TypeError(f"'level' must be 'int', received: '{type(level).__name__}'")
  if level < 1:
      raise TypeError(f"'level' must be greater than 0, received: {level}")
  
  mother_wavelet_matrix = ['haar', 'db1', 'db2', 'db3', 'db4', 'db5', 'db6', 'db7', 'db8', 'db9', 'db10', 'db11', 'db12', 'db13', 'db14', 'db15',
                            'db16', 'db17', 'db18', 'db19', 'db20', 'db21', 'db22', 'db23', 'db24', 'db25', 'db26', 'db27', 'db28', 'db29', 'db30', 'db31', 'db32', 'db33', 'db34',
                            'db35', 'db36', 'db37', 'db38', 'sym2', 'sym3', 'sym4', 'sym5', 'sym6', 'sym7', 'sym8', 'sym9', 'sym10', 'sym11', 'sym12', 'sym13', 'sym14', 'sym15', 'sym16', 'sym17',
                            'sym18', 'sym19', 'sym20', 'coif1', 'coif2', 'coif3', 'coif4', 'coif5', 'coif6', 'coif7', 'coif8', 'coif9', 'coif10', 'coif11', 'coif12', 'coif13', 'coif14', 'coif15', 'coif16', 'coif17',
                            'bior1.1', 'bior1.3', 'bior1.5', 'bior2.2', 'bior2.4', 'bior2.6', 'bior2.8', 'bior3.1', 'bior3.3', 'bior3.5', 'bior3.7', 'bior3.9', 'bior4.4', 'bior5.5', 'bior6.8', 'rbio1.1', 'rbio1.3', 'rbio1.5',
                            'rbio2.2', 'rbio2.4', 'rbio2.6', 'rbio2.8', 'rbio3.1', 'rbio3.3', 'rbio3.5', 'rbio3.7', 'rbio3.9', 'rbio4.4', 'rbio5.5', 'rbio6.8', 'dmey', 'gaus1', 'gaus2', 'gaus3', 'gaus4', 'gaus5', 'gaus6', 'gaus7',
                            'gaus8', 'mexh', 'morl', 'cgau1', 'cgau2', 'cgau3', 'cgau4', 'cgau5', 'cgau6', 'cgau7', 'cgau8', 'shan', 'fbsp', 'cmor']
  if mother_wavelet not in mother_wavelet_matrix:
      raise TypeError("'mother_wavelet' inválida, deve ser " + mother_wavelet_matrix + f" , recebido: '{mother_wavelet}'")

  original_len = len(signal)
  approx_coeff = []
  detail_coeff = []
  wavelet = Wavelet(mother_wavelet)
  low_filter = wavelet.dec_lo
  high_filter = wavelet.dec_hi
  filter_size = len(low_filter)

  try:
    #loop conforme os níveis de decomposição estipulados
    for _ in range(level):

      padded_signal = padding_symmetric(signal, filter_size)

      #aplicação dos filtros passa alta e passa baixa
      low_pass_filtered_signal = convolve(padded_signal, low_filter)[filter_size:(2*filter_size)+len(signal)-1] 
      low_pass_filtered_signal = low_pass_filtered_signal[1:len(low_pass_filtered_signal):2]
      high_pass_filtered_signal = convolve(padded_signal, high_filter)[filter_size:filter_size+len(signal)+filter_size-1]
      high_pass_filtered_signal = high_pass_filtered_signal[1:len(high_pass_filtered_signal):2]

      #armazenando os coeficientes detalhados e aproximados
      approx_coeff.append(low_pass_filtered_signal)
      detail_coeff.append(high_pass_filtered_signal)

      signal = low_pass_filtered_signal
  except:
    raise

  low_reconstruction_filter = wavelet.rec_lo
  high_reconstruction_filter = wavelet.rec_hi
  real_lengths = []

  for i in range(level-2,-1,-1):
    real_lengths.append(len(approx_coeff[i]))
  
  real_lengths.append(original_len)
  restored_approx_coeff = []

  for i in range(level):
    restored_signal = restore_signal(approx_coeff[i], low_reconstruction_filter, real_lengths[level-1-i])
    for j in range(i):
      restored_signal = restore_signal(restored_signal, low_reconstruction_filter, real_lengths[level-1-j])
    restored_approx_coeff.append(restored_signal)
  restored_detail_coeff = []

  for i in range(level):
      restored_signal = restore_signal(detail_coeff[i], high_reconstruction_filter, real_lengths[level-1-i])
      for j in range(i):
          restored_signal = restore_signal(restored_signal, high_reconstruction_filter, real_lengths[level-i-j])
      restored_detail_coeff.append(restored_signal)
  return restored_approx_coeff, restored_detail_coeff




In [ ]:
#CLASSE MAIN
from numpy.lib.twodim_base import tril_indices
import pandas as pd
import csv
import numpy as np
from numpy import array
import os, os.path
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA, FastICA
import matplotlib.pyplot as plt
from google.colab import drive
import os, os.path
import matplotlib.pyplot as plt

#realizando o mount com o google drive
drive.mount('/content/gdrive', force_remount=True)
gdrive_path = '/content/gdrive/MyDrive/Colab_Notebooks/TCC2_repository/DataSet_DENS/trials'

#definindo os valores de entrada time_windowing
time_window = [int(15124.5/250)]
frequency = 250
start_type = 'custom'

#dados inicias da dwt
level=5
mother_wavelet='sym6'


#importando o tempo de início de cada estímulo
stimulus_path = '/content/gdrive/MyDrive/Colab_Notebooks/TCC2_repository/estimulos/all_stimulus.csv'
time_file = pd.read_csv(f'{stimulus_path}', delimiter=';')
trial_data = array(pd.DataFrame(time_file, columns= ['trial'],dtype= str))
time_data = array(pd.DataFrame(time_file, columns= ['time'], dtype=float))
label_data = array(pd.DataFrame(time_file, columns= ['label'], dtype=str))



#lista e ordena os ensaios no diretório no google drive
trials = os.listdir(gdrive_path)
trials.sort()
print(trials)
print(len(trials))

#variável de iteração para controle dos cortes de cada amostra
i = 0
 
error_trials = []
matrix_guide = []
all_segmented_signals = []
truncated_segmented_signals = []



#loop para criar a matriz de cada sinal e segmentá-la de acordo com os eventos
for trial in trials:
  eeg_matrix = create_eeg_matrix(gdrive_path)
  print(50*"*")

  segmented_full = []
  #loop para segmentar os eventos
  for trial_value in trial_data:

    #condição para verificar quantos eventos cada ensaio poussi no arquivo EEG
    if int(trial_value) == int(trial.split('t')[-1]):
      time_stimulus = time_data[i]
      label_stimulus = label_data[i]
      i+=1
      print("Ensaio" + (trial.split('t')[-1]) + ": ")
      print(time_stimulus)

      recording_length = len(eeg_matrix[0,0])
      total_recording_time = recording_length/frequency
      
      #referência de ensaios com erro
      if(total_recording_time < time_stimulus+time_window):
        error_trial = (int(trial_value), str(label_stimulus) ,float(time_stimulus))
        error_trials.append(error_trial)
                        
      else:
        #armazena os eventos segmentados em ordem
        segmented_matrix = time_windowing(eeg_matrix, frequency, time_window, start_type, time_stimulus)
        guide = (int(trial_value), str(label_stimulus) ,float(time_stimulus))
        matrix_guide.append(guide)
        segmented_full.append(segmented_matrix)
  
  #armazena os grupos de eventos segmentados
  all_segmented_signals.append(segmented_full)

#salva matriz com sinais já segmentados no drive
np.save("/content/gdrive/MyDrive/Colab_Notebooks/TCC2_repository/resultados/all_segmented_signals.npy", all_segmented_signals)

#log de sucesso, retornando o formato das matrizes resultantes
print(f"Junção dos sinais full: {np.shape(all_segmented_signals)}")
print("Sucesso!")
print(f"Erro trials: {np.shape(error_trials)}")
print(f"Matrix_guide: {np.shape(matrix_guide)}")

#gerando arquivo de guia para os eventos que foram armazenados na matriz e seus respectivos tempos de início.
df = pd.DataFrame({'Trial': matrix_guide[0]})
error_matrix = np.matrix(error_trials)
print(matrix_guide)
pd.DataFrame(matrix_guide).to_csv("/content/gdrive/MyDrive/Colab_Notebooks/TCC2_repository/resultados/matrix_guide.csv")


normalized_approx = []
normalized_approx_coeff = []
normalized_approx_coeff2 = []

normalized_detail = []
normalized_detail3 = []
normalized_detail4 = []

stimulus_approx_coeff_matrix = []
stimulus_detail_coeff_matrix = []

trails_coeff = []

n=0
m=0
i=0
j=0
k=0

#iniciando método de normalização
scaler = MinMaxScaler()

#iniciando PCA
pca = PCA()

print(f"Junção dos sinais full *carregados*: {np.shape(all_segmented_signals)}")
print(f"i = {len(all_segmented_signals)}")
print(f'j = {len(all_segmented_signals[i])}')
print(f'k= {len(all_segmented_signals[i][j][0])}')


#loops aninhados para submeter todos os elementos da matriz à extração de características
for i in range(len(all_segmented_signals)):
  events_coeff = []
  for j in range(len(all_segmented_signals[i])):
    final_normalized_matrix = []
    for k in range(len(all_segmented_signals[i][j][0])):
      coeff_matrix = []
      coeff_matrix = []
      coeff_detail = []
      coeff_approx = []
      coeff_detail3 = []
      coeff_detail4 = []
      noarmalized_matrix = []

      signal = all_segmented_signals[i][j][0][k]

      #realizando a extração de características
      approx_coeff, detail_coeff = dwt(signal, level, mother_wavelet)

      #separando coeficientes de tamanhos diferentes
      coeff_matrix.append(detail_coeff[0])
      coeff_matrix.append(detail_coeff[1])
      coeff_detail.append(detail_coeff[2])
      coeff_detail3.append(detail_coeff[3])
      coeff_detail4.append(detail_coeff[4])
      coeff_approx.append(approx_coeff[4])

      
      #normalização e armazenamento dos coeficientes
      normalized_matrix = scaler.fit_transform(np.reshape(coeff_matrix, (-1,1)))
      normalized_matrix = np.reshape(normalized_matrix, (1,-1))

      normalized_detail = scaler.fit_transform(np.reshape(coeff_detail, (-1,1)))
      normalized_detail = np.reshape(normalized_detail, (1,-1))

      normalized_detail3 = scaler.fit_transform(np.reshape(coeff_detail3, (-1,1)))
      normalized_detail3 = np.reshape(normalized_detail3, (1,-1))

      normalized_detail4 = scaler.fit_transform(np.reshape(coeff_detail4, (-1,1)))
      normalized_detail4 = np.reshape(normalized_detail4, (1,-1))

      normalized_approx = scaler.fit_transform(np.reshape(coeff_approx, (-1,1)))
      normalized_approx = np.reshape(normalized_approx, (1,-1))  
      
      #agrupando todos os coeficientes (D1, D2, D3, D4, D5 E A5)
      normalized_matrix = np.append(normalized_matrix, normalized_detail)
      normalized_matrix = np.append(normalized_matrix, normalized_detail3)
      normalized_matrix = np.append(normalized_matrix, normalized_detail4)
      normalized_matrix = np.append(normalized_matrix, normalized_approx)

      

      #armazena coeficientes
      final_normalized_matrix.append(normalized_matrix)

    #laço j, seleção de características
    select_coeff = pca.fit_transform(final_normalized_matrix) 
    events_coeff.append(select_coeff)
    

  #laço i
  trails_coeff.append(events_coeff)

  #logs de execução
  print(i)
  print(f'Matrix final = {np.shape(trails_coeff)}')
  print(f'Primeira linha = {np.shape(trails_coeff[0])}') 



#log de resultados da extração de características
print(f'Método invocado: {n} vezes')
print(f'Número de blocos de 132 canais: {m}')
print(f'Número de trials: {i}' )

#armazenando a matriz com as características extraídas
np.save("/content/gdrive/MyDrive/Colab_Notebooks/TCC2_repository/resultados/trial_approx_coeff_matrix.npy",trails_coeff)


In [ ]:
#comparação entre coeficientes antes e depois da normalização
np.set_printoptions(threshold=np.sys.maxsize)
print(f'Coeficientes D4: {detail_coeff[4]} ')
print(f'Coeficientes D4 normalizados: {normalized_detail4}')


In [ ]:
#Geração de gráfico de distribuição de informação do PCA
per_var = np.round(pca.explained_variance_ratio_* 100, decimals=1)
print(per_var)
labels = ['PC' + str(x) for x in range(1, 11)]

plt.bar(x=range(1, 11), height=per_var[0:10], tick_label=labels)
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.ylabel('Porcentagem de Variância Percentual', fontsize = 18)
plt.xlabel('Valores Singulares', fontsize = 18)
plt.show()

In [ ]:
#geração de gráfico de decomposição
import matplotlib.pyplot as plt

all_segmented_signals = np.load('/content/gdrive/MyDrive/Colab_Notebooks/TCC2_repository/resultados/all_segmented_signals.npy', allow_pickle=True)
approx_coeff, detail_coeff = dwt(all_segmented_signals[3][0][0][120], 5, 'sym6')
plt.rcParams['figure.figsize'] = [20,10]
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.subplot(611)
plt.plot(range(len(detail_coeff[0])),detail_coeff[0], 'b')
plt.subplot(612)
plt.plot(range(len(detail_coeff[1])),detail_coeff[1], 'g')
plt.subplot(613)
plt.plot(range(len(detail_coeff[2])),detail_coeff[2], 'c')
plt.subplot(614)
plt.plot(range(len(detail_coeff[3])),detail_coeff[3], 'r')
plt.subplot(615)
plt.plot(range(len(detail_coeff[4])),detail_coeff[4], 'm')
plt.subplot(616)
plt.plot(range(len(approx_coeff[4])),approx_coeff[4], 'k')
plt.tight_layout()
plt.show()




In [ ]:
#Verificação de ensaios sem estímulos
from google.colab import drive
import numpy as np

drive.mount('/content/gdrive', force_remount=True)
all_segmented_signals = np.load('/content/gdrive/MyDrive/Colab_Notebooks/TCC2_repository/resultados/all_segmented_signals.npy', allow_pickle=True)
for i in range(len(all_segmented_signals)):
  print(f'{i} Tem conteúdo vazio: {all_segmented_signals[i]==[]}')

In [ ]:
#Geração de gráfico para comparação de estímulos sem tratamento nos sianis
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)
all_segmented_signals = np.load('/content/gdrive/MyDrive/Colab_Notebooks/TCC2_repository/resultados/all_segmented_signals.npy', allow_pickle=True)
plt.subplot(211)
plt.plot(range(len(all_segmented_signals[3][0][0][0])),all_segmented_signals[3][0][0][120], 'k')
plt.subplot(212)
plt.plot(range(len(all_segmented_signals[3][1][0][120])),all_segmented_signals[3][1][0][120], 'k')

In [ ]:
#Manipulação dos dados obtidos
from random import seed
from sklearn.svm import SVC
from numpy.lib.twodim_base import tril_indices
import pandas as pd
import csv
import numpy as np
from numpy import array
import os, os.path
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA, FastICA
from google.colab import drive
import os, os.path
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

event_test_X = []
event_X = []
X = []
y = []
y_name = []
y_events = []

#importando dados de etapas anteriores
drive.mount('/content/gdrive', force_remount=True)
raw_X = np.load('/content/gdrive/MyDrive/Colab_Notebooks/TCC2_repository/resultados/trial_approx_coeff_matrix.npy', allow_pickle=True)
events_classification = pd.read_csv('/content/gdrive/MyDrive/Colab_Notebooks/TCC2_repository/estimulos/stimulus_classification_new.csv')
matrixguide = pd.read_csv('/content/gdrive/MyDrive/Colab_Notebooks/TCC2_repository/resultados/matrix_guide.csv')
classificacao = pd.read_csv('/content/gdrive/MyDrive/Colab_Notebooks/TCC2_repository/estimulos/classificacao.csv')


events = array(pd.DataFrame(matrixguide, columns= ['1'],dtype= str))

#adequando o tamanho da matriz de resultados
for i in range(len(events)):
  y_events.append(events[i][0])


print(y_events)
print(f'events shape = {np.shape(events)}')

#label of dots
stimulus_name = array(pd.DataFrame(events_classification, columns= ['stimuliName'],dtype= str))


#valores de escala valência, excitação e dominância
valence = array(pd.DataFrame(events_classification, columns= ['valence'],dtype= float))
arousal = array(pd.DataFrame(events_classification, columns= ['arousal'],dtype= float ))
dominance = array(pd.DataFrame(events_classification, columns= ['dominance'],dtype= float ))
label = array(pd.DataFrame(classificacao, columns= ['label'], dtype= str))
print(pd.DataFrame(classificacao))

#loop para agrupar os eventos sem os ensaios
for i in range(len(raw_X)):
  for j in range(len(raw_X[i])):
      event_X.append(raw_X[i][j])

print(f'Event_X shape {np.shape(event_X)}')

#loop para popular X
for i in range(len(event_X)):
  z = []
  for j in range(len(event_X[i])):
    for k in range(len(event_X[i][j])):
      z.append(event_X[i][j][k])
  X.append(z)

print(np.ndim(valence))
print(np.shape(valence))

y_valence = []
y_arousal = []
y_dominance = []
#loop para popular y
for i in range(len(valence)):
  v = []
  a = []
  d = []
  v.append(valence[i][0])
  a.append(arousal[i][0])
  d.append(dominance[i][0])
  y_valence.append(v)
  y_arousal.append(a)
  y_dominance.append(d)


label_emotion = []

#ponto médio de cada escala
#valência
i = 5
#excitação
j = 5.1999999
#dominância
h = 4.0

#determinando as emoções com relação ao PAD
print(valence[0][0])
for k in range(len(valence)):
  if y_valence[k][0]>=i and y_arousal[k][0]>=j:
    if y_dominance[k][0]>=h:
      label_emotion.append("ALEGRE")
    else: 
      label_emotion.append("SURPRESO")
  elif y_valence[k][0]>=i and y_arousal[k][0]<j:
    if y_dominance[k][0]>=h:
      label_emotion.append("SATISFEITO")
    else:  
      label_emotion.append("PROTEGIDO")
  elif y_valence[k][0]<i and y_arousal[k][0]>=j:
    if y_dominance[k][0]>=h:
      label_emotion.append("DESPREOCUPADO")
    else:
      label_emotion.append("TRISTE")
  elif y_valence[k][0]<i and y_arousal[k][0]<j:
    if y_dominance[k][0]>=h:
      label_emotion.append("BRAVO")
    else:  
      label_emotion.append("MEDO")
print(f"i = {i} and j = {j}")
print(label_emotion)

#Dividindo os dados para teste e treino (proporção 80% treino e 20% teste)
treino_x, teste_x, treino_y, teste_y = train_test_split(X, label_emotion, 
                                                    test_size = 0.20, random_state=3)

print(f'treino_x shape = {np.shape(treino_x)}')
print(f'treino_y shape = {np.shape(treino_y)}')
print(f'teste_x shape = {np.shape(teste_x)}')
print(f'teste_y shape = {np.shape(teste_y)} ')

In [ ]:
#loop para descobrir melhor ponto intermediário
winner = []
accuracy_array = [0]
for i in np.arange(4.0,5.5,0.1):
  for j in np.arange(4.0,5.5,0.1):
    for h in np.arange(4.0,5.5,0.1):
      label_testing = []
      for k in range(len(valence)):
        if y_valence[k]>=i and y_arousal[k]>=j:
          if y_dominance[k]>=h:
            label_testing.append("ALEGRE")
          else: 
            label_testing.append("SURPRESO")
        elif y_valence[k]>=i and y_arousal[k]<j:
          if y_dominance[k]>=h:
            label_testing.append("SATISFEITO")
          else:  
            label_testing.append("PROTEGIDO")
        elif y_valence[k]<i and y_arousal[k]>=j:
          if y_dominance[k]>=h:
            label_testing.append("DESPREOCUPADO")
          else:
            label_testing.append("TRISTE")
        elif y_valence[k]<i and y_arousal[k]<j:
          if y_dominance[k]>=h:
            label_testing.append("BRAVO")
          else:  
            label_testing.append("MEDO")
      print(f"i = {i} and j = {j} and {h}")
      treino_x, teste_x, treino_y, teste_y = train_test_split(X, label_testing, 
                                                          test_size = 0.20, random_state=3)

      model = SVC()
      neigh = KNeighborsClassifier()

      #seting the training variables
      model.fit(treino_x,treino_y)
      prediction = model.predict(teste_x)

      #real prediction
      prediction.view()
      accuracy = accuracy_score(teste_y, prediction) * 100


      print(accuracy)
      print(max(accuracy_array))
      if(accuracy> max(accuracy_array)):
        print(f'Winner accuracy is: {accuracy}, with the medium valance value of {i}, arousal {j} and dominance {h}')
        winner = [accuracy, i, j]
        dummy_stratified = DummyClassifier()
        dummy_stratified.fit(treino_x, treino_y)
        previsoes = dummy_stratified.predict(teste_x)
        accuracy_dummy = accuracy_score(teste_y, previsoes) * 100
        print(f'Dummy classifier = {accuracy_dummy}')   
        results_svm = pd.DataFrame({'Prediction': prediction, 'Real value': teste_y})
        print(results_svm)
        winner_label = label_testing
      accuracy_array.append(accuracy)
print(winner)

In [ ]:
#knn e svm
from sklearn.metrics import confusion_matrix


print(f'X shape: {np.shape(X)}')
print(f'y_name shape: {np.shape(y_name)}')
print(f'y shape: {np.shape(y)}')
#print(f'y {array(y)}')



array(y).view()
array(teste_y).view()


model = SVC()
neigh = KNeighborsClassifier(n_neighbors=15)

#treino SVM
model.fit(treino_x,treino_y)
prediction = model.predict(teste_x)

#previsão SVM
prediction.view()
accuracy = accuracy_score(teste_y, prediction) * 100

#dummy prediction
dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
previsoes = dummy_stratified.predict(teste_x)
accuracy_dummy = accuracy_score(teste_y, previsoes) * 100



#KNN
neigh.fit(treino_x, treino_y)
prediction_knn = neigh.predict(teste_x)
accuracy_knn = accuracy_score(teste_y, prediction_knn) * 100

teste = confusion_matrix(teste_y, prediction)

#resultados da execução
print(f'Acurácia SVM = {accuracy}')
print(f'Acurácia KNN = {accuracy_knn}')
print(f'Acurácia dummy = {accuracy_dummy}')
results_svm = pd.DataFrame({'Prediction SVM': prediction, 'Prediction KNN': prediction_knn ,'Real value': teste_y})
print(results_svm)

In [11]:
#Matriz de confusão
def matrix_values(result_matrix, emotion):
  #iniciação dos valores derivados da matriz (falso negativo, falso positivo, verdadeiro negativo e verdadeiro positivo )
  fn = 0
  fp = 0
  vn = 0
  vp = 0
  index = np.where(labels == emotion)[0][0]
  vp = result_matrix[index,index]
  for i in range(len(result_matrix)):
    fn += result_matrix[index,i]
    fp += result_matrix[i,index]
  fn = fn - vp
  fp = fp - vp
  total = sum(sum(result_matrix))
  vn = total - fn - fp - vp

  #calculando métricas da matriz de confusão
  accuracy_matrix = (vp+vn)/total *100
  sensibility = vp/(vp + fn) *100
  precision = vp/(vp + fp) *100
  specificity = vn/(vn+fp) *100
  f1_score = 2*((precision*sensibility)/(precision+sensibility))
  print(f'{emotion}')
  print(f'vp = {vp}, vn = {vn}, fn = {fn} e fp = {fp}')
  print("Valores derivados da matriz")
  print(f'Acurácia = {accuracy_matrix:.2f}% \nSensibilidade = {sensibility:.2f}% \nPrecisão = {precision:.2f}% \nEspecificidade = {specificity:.2f}% \nF1-score = {f1_score:.2f} ')

In [ ]:
#Gerando matriz de confusão
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import seaborn as sns

result_matrix = confusion_matrix(teste_y, prediction, normalize='true')
labels= unique_labels(teste_y)
np.insert(labels, 6, 'SURPRESO')
print(labels)
colunas = [f'Previsto {label}' for label in labels]
indeces = [f'Real {label}' for label in labels]
table = pd.DataFrame(result_matrix, 
                      columns = colunas, index = indeces)
sns.heatmap(table, annot= True, fmt = '.1f', cmap='crest' )
for emotion in labels:
  matrix_values(result_matrix, emotion)

